In [11]:
!pip install 'requests[security]'

In [12]:
!pip install fake-useragent

In [3]:
import requests
import time
import re 
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import json
import pandas as pd

In [32]:
def deEmojify(string_unicode):
    string_encode = string_unicode.encode('ascii', 'ignore')
    string_decode = string_encode.decode()  
    return string_decode.replace('\r\n', '')

In [33]:
def extract_from_link(link):
    ua = UserAgent()

    headers =  {
      'Accept-Encoding': '*',
      'Connection': 'keep-alive'
    }
    
    headers['User-Agent'] = ua.random
    content = []
    host_name = 'https://www.lsa.umich.edu/cg/'

    print('extracting from :', host_name+link)
    
    try:
        page = requests.get(host_name + link, headers=headers)
        
    except requests.exceptions.ConnectionError:
    
        time.sleep(0.5)
        
    page = requests.get(host_name + link, headers = headers )

    soup = BeautifulSoup(page.content, 'html.parser')

    links = soup.find_all('div', {'class': 'ClassHyperlink'})

    for link2 in links:
        try:
            page_2 = requests.get(host_name + link2.get('data-url'), headers=headers)
            content.append(extract_course(BeautifulSoup(page_2.content, 'html.parser')))
            
        except requests.exceptions.ConnectionError:
            time.sleep(0.5)
            page_2 = requests.get(host_name + link2.get('data-url'), headers=headers)
            content.append(extract_course(BeautifulSoup(page_2.content, 'html.parser')))

    next = soup.find_all('a', {'id': 'contentMain_hlnkNextBtm'}, href=True)
    
    if len(next) > 0:
        content.extend(extract_from_link(next[0]['href']))

    return content

In [16]:
def split_table(data):

    data_dict = {}

    for d in data:
        split_data = deEmojify(d[0]).split(':')
    try:
        data_dict[split_data[0]] = split_data[1].strip()
    except:
        pass
    return data_dict

In [17]:
def extract_course(soup):

    table = soup.find('table')
    rows = table.findAll('tr')
    data = [[td.findChildren(text=True) for td in tr.findAll('td')] for tr in rows]
    data = [[u''.join(d).strip() for d in l] for l in data]
    data = split_table(data)

    title_id = {
      'title' : 'contentMain_lblLongTitle',
      'subject': 'contentMain_lblSubject',
      'catalog_number' :'contentMain_lblCatNo',
      'label_term': 'contentMain_lblTerm',
      'section': 'contentMain_lblSection',
      'sub_title': 'contentMain_lblSubtitle',
      'credits': 'contentMain_lblCredits',
      'requirements_distribution': 'contentMain_lblDist',
      'other': 'contentMain_lblOtherAttr',
      'consent': 'contentMain_lblConsent',
      'advisory_prerequisites': 'contentMain_lblAdvPre',
      'other_course_info': 'contentMain_lblOtherInfo',
      'repeatability': 'contentMain_lblRepeat'
    }

    for title in title_id:
        try: 
            data[title] = soup.find('span', {'id': title_id[title]}).text
            data['description'] = soup.find_all('span', {'id': 'contentMain_lblDescr'})[0].find('p').text
            data['primary_instructor'] = soup.find_all('div', {'class': 'classdetailsrow'})[-1].find('a').text 
    
        except AttributeError:
            pass

    return data

In [29]:
def get_data(url, file_name):
    content = []

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    links = soup.find_all('a', href=True)

    for i in range(11, len(links)-2, 2):
        while True:
            try:
                while True:
                    try:
                        link = links[i]['href']
                        time.sleep(0.1)
                        content.extend(extract_from_link(link))

                    except ConnectionError:
                        time.sleep(150)
                        continue

                    break
                    
            except ConnectionError:
                time.sleep(4000)
                continue
                
            break
        
    with open('data.json', 'w') as f:
        json.dump(content, f)
        
    df = pd.DataFrame(content)
    
    df.to_csv(file_name, index=False)
    
    return df

In [27]:
f_21 = get_data('https://www.lsa.umich.edu/cg/cg_subjectlist.aspx?termArray=f_21_2360&cgtype=ug&allsections=true', 'f_21_des.csv')

extracting from : https://www.lsa.umich.edu/cg/cg_results.aspx?termArray=f_21_2360&cgtype=ug&department=AAS&allsections=true&show=40
extracting from : https://www.lsa.umich.edu/cg/cg_results.aspx?termArray=f_21_2360&cgtype=ug&department=AAS&allsections=true&show=40&iPageNum=2
extracting from : https://www.lsa.umich.edu/cg/cg_results.aspx?termArray=f_21_2360&cgtype=ug&department=AERO&allsections=true&show=40
extracting from : https://www.lsa.umich.edu/cg/cg_results.aspx?termArray=f_21_2360&cgtype=ug&department=AEROSP&allsections=true&show=40
extracting from : https://www.lsa.umich.edu/cg/cg_results.aspx?termArray=f_21_2360&cgtype=ug&department=AES&allsections=true&show=40
extracting from : https://www.lsa.umich.edu/cg/cg_results.aspx?termArray=f_21_2360&cgtype=ug&department=ALA&allsections=true&show=40
extracting from : https://www.lsa.umich.edu/cg/cg_results.aspx?termArray=f_21_2360&cgtype=ug&department=ALA&allsections=true&show=40&iPageNum=2
extracting from : https://www.lsa.umich.edu

In [30]:
w_22 = get_data('https://www.lsa.umich.edu/cg/cg_subjectlist.aspx?termArray=w_22_2370&cgtype=ug&allsections=true', 'w_22_des.csv')

extracting from : https://www.lsa.umich.edu/cg/cg_results.aspx?termArray=w_22_2370&cgtype=ug&department=AAS&allsections=true&show=40
extracting from : https://www.lsa.umich.edu/cg/cg_results.aspx?termArray=w_22_2370&cgtype=ug&department=AAS&allsections=true&show=40&iPageNum=2
extracting from : https://www.lsa.umich.edu/cg/cg_results.aspx?termArray=w_22_2370&cgtype=ug&department=AERO&allsections=true&show=40
extracting from : https://www.lsa.umich.edu/cg/cg_results.aspx?termArray=w_22_2370&cgtype=ug&department=AEROSP&allsections=true&show=40
extracting from : https://www.lsa.umich.edu/cg/cg_results.aspx?termArray=w_22_2370&cgtype=ug&department=AES&allsections=true&show=40
extracting from : https://www.lsa.umich.edu/cg/cg_results.aspx?termArray=w_22_2370&cgtype=ug&department=ALA&allsections=true&show=40
extracting from : https://www.lsa.umich.edu/cg/cg_results.aspx?termArray=w_22_2370&cgtype=ug&department=ALA&allsections=true&show=40&iPageNum=2
extracting from : https://www.lsa.umich.edu

In [11]:
def create_course_col(df):

    df['Catalog Nbr'] = df['catalog_number'].dropna().astype(int).astype(str)
    df['course'] = df['subject'] + ' ' + df['Catalog Nbr']
    
    return df

In [12]:
w_22 = pd.read_csv('w_22_des.csv')
f_21 = pd.read_csv('f_21_des.csv')

In [13]:
w_22 = create_course_col(w_22)
w_22.to_csv('w_22_des.csv', index=False)

In [14]:
f_21 = create_course_col(f_21)
f_21.to_csv('f_21_des.csv', index=False)